# PolrGWAS.jl

PolrGWAS.jl is a Julia package for performing genome-wide association studies for ordered categorical phenotypes. 

This package requires Julia v0.7.0 or later. The package has not yet been registered and must be installed using the repository location. Start julia and use the ] key to switch to the package manager REPL
```julia
(v0.7) pkg> add https://github.com/Hua-Zhou/PolrGWAS.git#juliav0.7
```

In [1]:
versioninfo()

Julia Version 0.7.0
Commit a4cb80f3ed (2018-08-08 06:46 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


In [2]:
using BenchmarkTools, CSV, PolrGWAS, SnpArrays

┌ Info: Recompiling stale cache file /Users/huazhou/.julia/compiled/v0.7/PolrGWAS/xQi4D.ji for PolrGWAS [00428148-03d9-50ae-bfb7-4a690d5612f1]
└ @ Base loading.jl:1185
┌ Warning: Package PolrGWAS does not have Distributions in its dependencies:
│ - If you have PolrGWAS checked out for development and have
│   added Distributions as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PolrGWAS
└ Loading Distributions into PolrGWAS from project dependency, future warnings for PolrGWAS are suppressed.


## Basic usage

Suppose covariates and phenotype are available in a csv file `covariate.txt`. Variable `trait` is the ordered categorical phenotypes coded as integers 1 to 4. We want to include variable `sex` as the covariate in GWAS.

In [3]:
;head -20 ../data/covariate.txt

famid,perid,faid,moid,sex,trait
2431,NA19916,0,0,1,4
2424,NA19835,0,0,2,4
2469,NA20282,0,0,2,4
2368,NA19703,0,0,1,3
2425,NA19901,0,0,2,3
2427,NA19908,0,0,1,4
2430,NA19914,0,0,2,4
2470,NA20287,0,0,2,1
2436,NA19713,0,0,2,3
2426,NA19904,0,0,1,1
2431,NA19917,0,0,2,1
2436,NA19982,0,0,1,2
2487,NA20340,0,0,1,4
2427,NA19909,0,0,2,4
2424,NA19834,0,0,1,4
2480,NA20317,0,0,2,4
2418,NA19818,0,0,1,1
2490,NA20346,0,0,1,2
2433,NA19921,0,0,2,4


Genotype data is available as binary Plink files.

In [4]:
;ls -l ../data/hapmap3.bed ../data/hapmap3.bim ../data/hapmap3.fam

-rw-r--r--  1 huazhou  staff  1128171 Sep  4 12:03 ../data/hapmap3.bed
-rw-r--r--  1 huazhou  staff   388672 Sep  4 12:03 ../data/hapmap3.bim
-rw-r--r--  1 huazhou  staff     7136 Sep  4 12:03 ../data/hapmap3.fam


There are 324 samples at 13,928 SNPs.

In [5]:
size(SnpArray("../data/hapmap3.bed"))

(324, 13928)

The following command performs GWAS using the proportional odds logistic regression.

In [6]:
polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3")

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1    -1.48564  0.358713 -4.14157    <1e-4
θ2   -0.569479  0.340649 -1.67175   0.0956
θ3    0.429815  0.339266   1.2669   0.2061
β1    0.424656  0.213911   1.9852   0.0480


In [7]:
@btime(polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", verbose=false))

  134.652 ms (764094 allocations: 35.47 MiB)


## Output files

`polrgwas` outputs two files: `polrgwas.nullmodel.txt` and `polrgwas.scoretest.txt`. The prefix `polrgwas` can be changed by the `outfile` keyword.

* `polrgwas.nullmodel.txt` lists the estimated regression model.  

In [8]:
;cat polrgwas.nullmodel.txt

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1    -1.48564  0.358713 -4.14157    <1e-4
θ2   -0.569479  0.340649 -1.67175   0.0956
θ3    0.429815  0.339266   1.2669   0.2061
β1    0.424656  0.213911   1.9852   0.0480


* `polrgwas.score.txt` lists the SNPs and their pvalues. 

In [9]:
;head polrgwas.scoretest.txt

chr,pos,snpid,maf,pval
1,554484,rs10458597,0.0,1.0
1,758311,rs12562034,0.07763975155279501,0.003001230754791864
1,967643,rs2710875,0.32407407407407407,2.5117214960313984e-5
1,1168108,rs11260566,0.19158878504672894,1.1373112253090032e-5
1,1375074,rs1312568,0.441358024691358,0.008317358366815329
1,1588771,rs35154105,0.0,1.0
1,1789051,rs16824508,0.00462962962962965,0.5274428530031907
1,1990452,rs2678939,0.4537037037037037,0.29988429741740025
1,2194615,rs7553178,0.22685185185185186,0.16436415589171904


In [10]:
rm("polrgwas.scoretest.txt")
rm("polrgwas.nullmodel.txt")

## Link functions

The `link` keyword argument of `polrgwas` can take value `LogitLink()` (default), `ProbitLink()` (ordred Probit model), or `CLoglogLink()` (proportional hazards model).

E.g., to perform GWAS using the ordred Probit model

In [11]:
polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", link=ProbitLink(), outfile="op")

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,ProbitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.866156  0.210746 -4.10995    <1e-4
θ2   -0.359878   0.20552 -1.75106   0.0809
θ3    0.247054  0.205135  1.20435   0.2293
β1    0.251058  0.128212  1.95814   0.0511


In [12]:
;cat op.nullmodel.txt

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,ProbitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.866156  0.210746 -4.10995    <1e-4
θ2   -0.359878   0.20552 -1.75106   0.0809
θ3    0.247054  0.205135  1.20435   0.2293
β1    0.251058  0.128212  1.95814   0.0511


In [13]:
;head op.scoretest.txt

chr,pos,snpid,maf,pval
1,554484,rs10458597,0.0,1.0
1,758311,rs12562034,0.07763975155279501,0.006450425919141508
1,967643,rs2710875,0.32407407407407407,1.5785042548448054e-5
1,1168108,rs11260566,0.19158878504672894,4.979075119251739e-6
1,1375074,rs1312568,0.441358024691358,0.004566574021452654
1,1588771,rs35154105,0.0,1.0
1,1789051,rs16824508,0.00462962962962965,0.4819721449163142
1,1990452,rs2678939,0.4537037037037037,0.33240414006705765
1,2194615,rs7553178,0.22685185185185186,0.2529908594647623


In [14]:
rm("op.scoretest.txt")
rm("op.nullmodel.txt")

## SNP and/or sample masks

In practice, we often want to perform GWAS on selected SNPs and/or selected samples. They can be specified by the `colinds` and `rowinds` keywords of `polrgwas` function.

For example, to perform GWAS on SNPs with minor allele frequency (MAF) above 0.05

In [15]:
snpinds = maf(SnpArray("../data/hapmap3.bed")) .≥ 0.05

13928-element BitArray{1}:
 false
  true
  true
  true
  true
 false
 false
  true
  true
  true
 false
  true
  true
     ⋮
  true
  true
  true
  true
  true
  true
  true
  true
  true
 false
 false
 false

In [16]:
@time polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", colinds = snpinds, outfile="commonvariant")

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1    -1.48564  0.358713 -4.14157    <1e-4
θ2   -0.569479  0.340649 -1.67175   0.0956
θ3    0.429815  0.339266   1.2669   0.2061
β1    0.424656  0.213911   1.9852   0.0480
  0.202850 seconds (801.70 k allocations: 37.926 MiB, 3.11% gc time)


In [17]:
;cat commonvariant.nullmodel.txt

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1    -1.48564  0.358713 -4.14157    <1e-4
θ2   -0.569479  0.340649 -1.67175   0.0956
θ3    0.429815  0.339266   1.2669   0.2061
β1    0.424656  0.213911   1.9852   0.0480


In [18]:
;head -20 commonvariant.scoretest.txt

chr,pos,snpid,maf,pval
1,758311,rs12562034,0.07763975155279501,0.003001230754791864
1,967643,rs2710875,0.32407407407407407,2.5117214960313984e-5
1,1168108,rs11260566,0.19158878504672894,1.1373112253090032e-5
1,1375074,rs1312568,0.441358024691358,0.008317358366815329
1,1990452,rs2678939,0.4537037037037037,0.29988429741740025
1,2194615,rs7553178,0.22685185185185186,0.16436415589171904
1,2396747,rs13376356,0.1448598130841121,0.5372089713885594
1,2823603,rs1563468,0.4830246913580247,0.23123684490822363
1,3025087,rs6690373,0.2538699690402477,0.7000923664008778
1,3431124,rs12093117,0.1099071207430341,0.4271132018338374
1,3633945,rs10910017,0.22187500000000004,0.9141485352635614
1,4096895,rs6702633,0.4752321981424149,0.006373000780228048
1,4297388,rs684965,0.3055555555555556,0.09402646589417124
1,4498133,rs11809295,0.0993788819875776,0.0856953578572361
1,4698713,rs578528,0.32407407407407407,0.06883563182592002
1,4899946,rs4654471,0.3580246913580247,0.2267196199789601
1,5100369,rs6681148,0.131

In [19]:
# headline in commonvariant.scoretest.txt
@show countlines("commonvariant.scoretest.txt"), count(snpinds)

(countlines("commonvariant.scoretest.txt"), count(snpinds)) = (12086, 12085)


(12086, 12085)

In [20]:
rm("commonvariant.scoretest.txt")
rm("commonvariant.nullmodel.txt")

User should be particularly careful when using the `rowinds` keyword. Selected rows in SnpArray should exactly match the samples in the null model. Otherwise the results are meaningless.

## Likelihood ratio test (LRT)

By default, `polrgwas` calculates the score test p-value for each SNP. Score test is fast because it doesn't require fitting alternative model. User can request likelihood ratio test (LRT) using keyword `test=:LRT`. LRT is much slower but may be more powerful than score test.

In [21]:
@time polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", test=:LRT, outfile="lrt")

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: trait ~ +sex

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1    -1.48564  0.358713 -4.14157    <1e-4
θ2   -0.569479  0.340649 -1.67175   0.0956
θ3    0.429815  0.339266   1.2669   0.2061
β1    0.424656  0.213911   1.9852   0.0480
 20.710309 seconds (8.81 M allocations: 2.065 GiB, 1.74% gc time)


Test result is output to `outfile.lrttest.txt` file

In [22]:
;head -20 lrt.lrttest.txt

chr,pos,snpid,maf,pval
1,554484,rs10458597,0.0,1.0
1,758311,rs12562034,0.07763975155279501,0.0019185836579804134
1,967643,rs2710875,0.32407407407407407,1.805050556976133e-5
1,1168108,rs11260566,0.19158878504672894,5.8733847126852225e-6
1,1375074,rs1312568,0.441358024691358,0.008081022577831297
1,1588771,rs35154105,0.0,1.0
1,1789051,rs16824508,0.00462962962962965,0.5169027130128576
1,1990452,rs2678939,0.4537037037037037,0.29946402200910055
1,2194615,rs7553178,0.22685185185185186,0.16151069094439868
1,2396747,rs13376356,0.1448598130841121,0.5387338201469207
1,2623158,rs28753913,0.0,1.0
1,2823603,rs1563468,0.4830246913580247,0.2312300208157546
1,3025087,rs6690373,0.2538699690402477,0.6995722170701131
1,3225416,rs12043519,0.029320987654321007,0.002016744167744886
1,3431124,rs12093117,0.1099071207430341,0.43052012312944177
1,3633945,rs10910017,0.22187500000000004,0.9142024828415289
1,3895935,rs34770924,0.024691358024691357,0.9980482910119114
1,4096895,rs6702633,0.4752321981424149,0.00630524

In [23]:
rm("lrt.lrttest.txt")
rm("lrt.nullmodel.txt")

In this case, GWAS by score test takes less than 0.2 second, while GWAS by LRT takes about 20 seconds. About 100 fold difference in run time. 

## Score for screening, LRT for power 

For large data sets, a practical solution is to perform score test first, then re-do LRT for the most promising SNPs according to score test p-values.

Step 1: Perform score test GWAS, results in hapmap.scoretest.txt

In [24]:
@time polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", test=:score, outfile="hapmap", verbose=false)

  0.231970 seconds (828.05 k allocations: 38.879 MiB, 11.88% gc time)


In [25]:
;head -20 hapmap.scoretest.txt

chr,pos,snpid,maf,pval
1,554484,rs10458597,0.0,1.0
1,758311,rs12562034,0.07763975155279501,0.003001230754791864
1,967643,rs2710875,0.32407407407407407,2.5117214960313984e-5
1,1168108,rs11260566,0.19158878504672894,1.1373112253090032e-5
1,1375074,rs1312568,0.441358024691358,0.008317358366815329
1,1588771,rs35154105,0.0,1.0
1,1789051,rs16824508,0.00462962962962965,0.5274428530031907
1,1990452,rs2678939,0.4537037037037037,0.29988429741740025
1,2194615,rs7553178,0.22685185185185186,0.16436415589171904
1,2396747,rs13376356,0.1448598130841121,0.5372089713885594
1,2623158,rs28753913,0.0,1.0
1,2823603,rs1563468,0.4830246913580247,0.23123684490822363
1,3025087,rs6690373,0.2538699690402477,0.7000923664008778
1,3225416,rs12043519,0.029320987654321007,0.001067589332379543
1,3431124,rs12093117,0.1099071207430341,0.4271132018338374
1,3633945,rs10910017,0.22187500000000004,0.9141485352635614
1,3895935,rs34770924,0.024691358024691357,0.9989137356071247
1,4096895,rs6702633,0.4752321981424149,0.00637300

Step 2: Sort score test p-values and find top 10 SNPs

In [26]:
scorepvals = CSV.read("hapmap.scoretest.txt")[5]

13928-element Array{Union{Missing, Float64},1}:
 1.0                  
 0.003001230754791864 
 2.5117214960313984e-5
 1.1373112253090032e-5
 0.008317358366815329 
 1.0                  
 0.5274428530031907   
 0.29988429741740025  
 0.16436415589171904  
 0.5372089713885594   
 1.0                  
 0.23123684490822363  
 0.7000923664008778   
 ⋮                    
 0.4533525658437985   
 0.6409263039143129   
 0.1581988881808424   
 0.5643246521154555   
 0.5016172691334395   
 0.1646704927506094   
 0.6977871465130079   
 0.9120338117720317   
 0.6977871465130516   
 0.31367403711981456  
 0.24779832332240803  
 0.24848633880732612  

In [27]:
tophits = sortperm(scorepvals)[1:10]

10-element Array{Int64,1}:
  6063
 13544
  5071
  2458
     4
  3291
 13737
  4183
     3
 12727

In [28]:
scorepvals[tophits]

10-element Array{Union{Missing, Float64},1}:
 1.4416637280589893e-6
 5.127005866008025e-6 
 5.9108456351498095e-6
 8.756466364523816e-6 
 1.1373112253090032e-5
 1.7228784857111697e-5
 2.0089089384105136e-5
 2.2839097761623567e-5
 2.5117214960313984e-5
 2.949378996752777e-5 

Step 3: Re-do LRT on top hits.

In [29]:
@time polrgwas(@formula(trait ~ 0 + sex), "../data/covariate.txt", "../data/hapmap3", colinds=tophits, test=:LRT, outfile="hapmap", verbose=false)

  0.174450 seconds (284.38 k allocations: 15.332 MiB, 4.35% gc time)


In [30]:
;head -20 hapmap.lrttest.txt

chr,pos,snpid,maf,pval
1,967643,rs2710875,0.32407407407407407,1.805050556976133e-5
1,1168108,rs11260566,0.19158878504672894,5.8733847126852225e-6
3,36821790,rs4678553,0.23456790123456794,1.1303825016262592e-5
4,11017683,rs16881446,0.27554179566563464,1.1105427468799613e-5
5,3739190,rs12521166,0.0679012345679012,4.7812882296576845e-5
6,7574576,rs1885466,0.17746913580246915,7.272346896740631e-6
7,41152376,rs28880,0.3379629629629629,9.180126530294943e-7
20,39225952,rs2076145,0.04475308641975306,5.94831595593157e-5
23,81247423,rs5923282,0.0030864197530864335,0.0016846467294181053
23,121048059,rs1937165,0.4380804953560371,1.9754643855527356e-5


In [31]:
rm("hapmap.lrttest.txt")
rm("hapmap.scoretest.txt")
rm("hapmap.nullmodel.txt")